In [1]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import joblib
import numpy as np
import pandas as pd
import time

In [2]:
df = pd.read_parquet("../data/processed/bluetooth.parquet")
df.head()

,Delta,Length,Protocol_HCI_EVT,Protocol_HCI_CMD,Protocol_HCI H4 Broadcom,Protocol_Baseband,Protocol_LMP,Protocol_L2CAP,Protocol_SDP,Protocol_BNEP,Protocol_RFCOMM,Protocol_OBEX,is_attack
0,0.005306,9,1,0,0,0,0,0,0,0,0,0,1
1,0.005306,10,0,1,0,0,0,0,0,0,0,0,1
2,0.005306,9,1,0,0,0,0,0,0,0,0,0,1
3,0.002642,13,1,0,0,0,0,0,0,0,0,0,1
4,0.002882,10,0,1,0,0,0,0,0,0,0,0,1


In [3]:
X = df.drop("is_attack", axis='columns')
y = df["is_attack"]
X.head(1)

,Delta,Length,Protocol_HCI_EVT,Protocol_HCI_CMD,Protocol_HCI H4 Broadcom,Protocol_Baseband,Protocol_LMP,Protocol_L2CAP,Protocol_SDP,Protocol_BNEP,Protocol_RFCOMM,Protocol_OBEX
0,0.005306,9,1,0,0,0,0,0,0,0,0,0


In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
start_time = time.time()
param_grid = {
    'pca__n_components': [2,3,4,5],
    'model__n_neighbors': np.arange(1, 10),  
    'model__weights': ['uniform', 'distance'], 
    'model__algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],  
    'model__leaf_size': np.arange(10, 51, 10),  
    'model__p': [1, 2] 
}

model = KNeighborsClassifier()

steps = [
    ('scaler', StandardScaler()),  
    ('pca', PCA()),  
    ('model', model) 
]

pipeline = Pipeline(steps)

grid_search = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=StratifiedKFold(n_splits=5), n_jobs=-1, verbose=1, scoring='f1')

grid_search.fit(X_train, y_train) 
end_time = time.time()
execution_time = end_time - start_time

best_params = grid_search.best_params_
best_score = grid_search.best_score_

best_model = grid_search.best_estimator_

print("Best Hyperparameters:", best_params)
print("Best F1 Score:", best_score)
print("Total Training Time kNN: {:.4f} seconds".format(execution_time))
print("Saving model")


joblib.dump(best_model, '../models/Bluetooth_attack/best_knn_model.pkl')
del grid_search

Fitting 5 folds for each of 2880 candidates, totalling 14400 fits


Best Hyperparameters: {'model__algorithm': 'brute', 'model__leaf_size': 10, 'model__n_neighbors': 7, 'model__p': 1, 'model__weights': 'distance', 'pca__n_components': 4}
Best F1 Score: 0.9376606539698458
Total Training Time kNN: 142.0097 seconds
Saving model


In [6]:
start_time = time.time()
param_grid = {
    'pca__n_components': [2,3,4,5],
    'model__n_estimators': [100, 200],
    'model__max_depth': [None, 5, 10],
    'model__min_samples_split': [2, 5],
    'model__min_samples_leaf': [1, 2],
    'model__max_features': ['auto', 'sqrt'],
}

model = RandomForestClassifier()

steps = [
    ('scaler', StandardScaler()),  
    ('pca', PCA()),  
    ('model', model) 
]

pipeline = Pipeline(steps)

grid_search = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=StratifiedKFold(n_splits=5), n_jobs=-1, verbose=1, scoring='f1')

grid_search.fit(X_train, y_train)

end_time = time.time()
execution_time = end_time - start_time


best_params = grid_search.best_params_
best_score = grid_search.best_score_

best_model = grid_search.best_estimator_

print("Best Hyperparameters:", best_params)
print("Best F1 Score:", best_score)
print("Total Training Time Random Forest: {:.4f} seconds".format(execution_time))
print("Saving model")


joblib.dump(best_model, '../models/Bluetooth_attack/best_rf_model.pkl')
del grid_search

Fitting 5 folds for each of 192 candidates, totalling 960 fits


/home/reikia/anaconda3/envs/py10/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
480 fits failed out of a total of 960.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
171 fits failed with the following error:
Traceback (most recent call last):
  File "/home/reikia/anaconda3/envs/py10/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/reikia/anaconda3/envs/py10/lib/python3.10/site-packages/sklearn/base.py", line 1151, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/home/reikia/anaconda3/envs/py10/lib/python3.10/site-packages/sklearn/pipeline.py", line 420, in fit


Best Hyperparameters: {'model__max_depth': 10, 'model__max_features': 'sqrt', 'model__min_samples_leaf': 1, 'model__min_samples_split': 2, 'model__n_estimators': 100, 'pca__n_components': 3}
Best F1 Score: 0.9413244937409955
Total Training Time Random Forest: 48.3065 seconds
Saving model


In [7]:
start_time = time.time()
param_grid = {
    'pca__n_components': [2,3,4,5],
    'model__learning_rate': [0.1, 0.01],
    'model__n_estimators': [100, 150],
    'model__max_depth': [2, 3, 5],
}

model = xgb.XGBClassifier()

steps = [
    ('scaler', StandardScaler()),  
    ('pca', PCA()),  
    ('model', model) 
]

pipeline = Pipeline(steps)

grid_search = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=StratifiedKFold(n_splits=5), n_jobs=-1, verbose=1, scoring='f1')

grid_search.fit(X_train, y_train)

end_time = time.time()
execution_time = end_time - start_time


best_params = grid_search.best_params_
best_score = grid_search.best_score_

best_model = grid_search.best_estimator_

print("Best Hyperparameters:", best_params)
print("Best F1 Score:", best_score)
print("Total Training Time XGBoost: {:.4f} seconds".format(execution_time))
print("Saving model")

joblib.dump(best_model, '../models/Bluetooth_attack/best_xgb_model.pkl')
del grid_search

Fitting 5 folds for each of 48 candidates, totalling 240 fits
Best Hyperparameters: {'model__learning_rate': 0.1, 'model__max_depth': 5, 'model__n_estimators': 150, 'pca__n_components': 5}
Best F1 Score: 0.9406835906899366
Total Training Time XGBoost: 3.9281 seconds
Saving model


In [8]:
start_time = time.time()

param_grid = {
    'pca__n_components': [2,3,4,5],
    'model__learning_rate': [0.05, 0.1],
    'model__n_estimators': [50, 100],
    'model__num_leaves': [10, 15],
    'model__max_depth': [4, 5],
}

model = lgb.LGBMClassifier()

steps = [
    ('scaler', StandardScaler()),  
    ('pca', PCA()),  
    ('model', model) 
]

pipeline = Pipeline(steps)

grid_search = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=StratifiedKFold(n_splits=5), n_jobs=-1, verbose=1, scoring='f1')

grid_search.fit(X_train, y_train)

end_time = time.time()
execution_time = end_time - start_time


best_params = grid_search.best_params_
best_score = grid_search.best_score_

best_model = grid_search.best_estimator_

print("Best Hyperparameters:", best_params)
print("Best F1 Score:", best_score)
print("Total Training Time LGBM: {:.4f} seconds".format(execution_time))
print("Saving model")

joblib.dump(best_model, '../models/Bluetooth_attack/best_lgb_model.pkl')
del grid_search

Fitting 5 folds for each of 64 candidates, totalling 320 fits
[LightGBM] [Info] Number of positive: 1738, number of negative: 4022
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000536 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 510
[LightGBM] [Info] Number of data points in the train set: 5760, number of used features: 2
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.301736 -> initscore=-0.839044
[LightGBM] [Info] Start training from score -0.839044
[LightGBM] [Info] Number of positive: 1738, number of negative: 4023
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000701 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 764
[LightGBM] [Info] Number of data points in the train set: 5761, number of used features: 3
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.301684 -> initscore=-0.839293
[LightGBM] [In

In [5]:
dic_models = {
    "kNN": {
        "path": "../models/Bluetooth_attack/best_knn_model.pkl"
    },
    "Random Forest": {
        "path": "../models/Bluetooth_attack/best_rf_model.pkl"
    },
    "XGBoost": {
        "path": "../models/Bluetooth_attack/best_xgb_model.pkl"
    },
    "LGBM": {
        "path": "../models/Bluetooth_attack/best_lgb_model.pkl"
    }
}

In [6]:
for model_name in dic_models.keys():
    model= joblib.load(dic_models[model_name]["path"])
    dic_models[model_name]["model"] = model

    y_pred = model.predict(X_test)

    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_pred)

    conf_matrix = confusion_matrix(y_test, y_pred)
    tn, fp, fn, tp = conf_matrix.ravel()
    tpr = tp / (tp + fn)
    fpr = fp / (fp + tn)

    print(model_name)
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1-score:", f1)
    print("AUC:", auc)
    print("TPR:", tpr)
    print("FPR:", fpr)
    print()
    dic_models[model_name]["Precision"] = precision
    dic_models[model_name]["recall"] = recall
    dic_models[model_name]["f1"] = f1
    dic_models[model_name]["auc"] = auc
    dic_models[model_name]["TPR"] = tpr
    dic_models[model_name]["FPR"] = fpr

kNN
Precision: 0.9320557491289199
Recall: 0.9553571428571429
F1-score: 0.9435626102292769
AUC: 0.9619654368596754
TPR: 0.9553571428571429
FPR: 0.031426269137792104

Random Forest
Precision: 0.9385964912280702
Recall: 0.9553571428571429
F1-score: 0.9469026548672567
AUC: 0.9635770404052031
TPR: 0.9553571428571429
FPR: 0.028203062046736505

XGBoost
Precision: 0.9587073608617595
Recall: 0.9535714285714286
F1-score: 0.9561324977618622
AUC: 0.9675189938989296
TPR: 0.9535714285714286
FPR: 0.018533440773569703

LGBM
Precision: 0.9381625441696113
Recall: 0.9482142857142857
F1-score: 0.9431616341030196
AUC: 0.9600056118337746
TPR: 0.9482142857142857
FPR: 0.028203062046736505



In [7]:
class Survey:
    def __init__(self, list_models):
        if len(list_models) % 2 == 0:
            return
        self.list_models = list_models

    def predict(self, X):
        ans = [0 for ___ in range(len(X))]
        for model in self.list_models:
            preds = model.predict(X)
            for i in range(len(preds)):
                ans[i] += preds[i]
        min_votes = len(self.list_models) // 2 + 1
        for i in range(len(ans)):
            if ans[i] >= min_votes:
                ans[i] = 1
            else:
                ans[i] = 0
        return ans

In [8]:
knn = dic_models["kNN"]["model"]
rf = dic_models["Random Forest"]["model"]
XGBoost = dic_models["XGBoost"]["model"]
LGBM = dic_models["LGBM"]["model"]
survey = Survey([rf, XGBoost, knn])

In [9]:
joblib.dump(survey, "../models/Bluetooth_attack/survey.pkl")

dic_models["Survey"] = {
    "path": "../models/Bluetooth_attack/survey.pkl",
    "model": survey
}

In [10]:
y_pred = survey.predict(X_test)

precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
tn, fp, fn, tp = conf_matrix.ravel()
tpr = tp / (tp + fn)
fpr = fp / (fp + tn)
print("Survey")



print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)
print("AUC:", auc)
print("TPR:", tpr)
print("FPR:", fpr)
print()
dic_models["Survey"]["Precision"] = precision
dic_models["Survey"]["recall"] = recall
dic_models["Survey"]["f1"] = f1
dic_models["Survey"]["auc"] = auc
dic_models["Survey"]["TPR"] = tpr
dic_models["Survey"]["FPR"] = fpr

Survey
Precision: 0.9417989417989417
Recall: 0.9535714285714286
F1-score: 0.9476486246672582
AUC: 0.96348998503511
TPR: 0.9535714285714286
FPR: 0.0265914585012087



In [11]:
joblib.dump(dic_models, "../models/Bluetooth_attack/all_models_and_results.pkl")

['../models/Bluetooth_attack/all_models_and_results.pkl']

In [12]:
model_name_list = list()
precision_list = list()
recall_list = list()
f1_list = list()
auc_list = list()
tpr_list = list()
fpr_list = list()

for model_name in dic_models.keys():
    model_name_list.append(model_name)
    precision_list.append(dic_models[model_name]["Precision"])
    recall_list.append(dic_models[model_name]["recall"])
    f1_list.append(dic_models[model_name]["f1"])
    auc_list.append(dic_models[model_name]["auc"])
    tpr_list.append(dic_models[model_name]['TPR'])
    fpr_list.append(dic_models[model_name]['FPR'])

results = pd.DataFrame({
    "Model Name": model_name_list,
    "Precision": precision_list,
    "Recall": recall_list,
    "F1": f1_list,
    "AUC": auc_list,
    "TPR": tpr_list,
    "FPR": fpr_list
})
results

,Model Name,Precision,Recall,F1,AUC,TPR,FPR
0,kNN,0.932056,0.955357,0.943563,0.961965,0.955357,0.031426
1,Random Forest,0.938596,0.955357,0.946903,0.963577,0.955357,0.028203
2,XGBoost,0.958707,0.953571,0.956132,0.967519,0.953571,0.018533
3,LGBM,0.938163,0.948214,0.943162,0.960006,0.948214,0.028203
4,Survey,0.941799,0.953571,0.947649,0.963490,0.953571,0.026591


In [13]:
results.to_csv("../results/Results_Bluetooth.csv", index=False)